In [20]:
import base64
import requests
import logging
import glob
import os
from langchain import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS, VectorStore
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.schema import Document
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
# Function to encode the image
def encode_image(image_path):
    logger.info(f"Encoding image from path: {image_path}")
    with open(image_path, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('utf-8')
    logger.info("Image encoding complete")
    return encoded_image

In [5]:
# Directory containing images
image_dir = "/Users/akileshjayakumar/Documents/catalystX2024/notebooks/mm_rag_images"

In [7]:
# Get all image files in the directory
image_paths = glob.glob(os.path.join(image_dir, "*.jpg"))
print(image_paths)

['/Users/akileshjayakumar/Documents/catalystX2024/notebooks/mm_rag_images/test1.jpg', '/Users/akileshjayakumar/Documents/catalystX2024/notebooks/mm_rag_images/test3.jpg', '/Users/akileshjayakumar/Documents/catalystX2024/notebooks/mm_rag_images/test2.jpg']


In [25]:
# Initialize FAISS vector store for image embeddings
embedding_model = OpenAIEmbeddings()

client=<openai.resources.embeddings.Embeddings object at 0x10f1cbc80> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x10f1b5b80> model='text-embedding-ada-002' deployment='text-embedding-ada-002' openai_api_version='' openai_api_base=None openai_api_type='' openai_proxy='' embedding_ctx_length=8191 openai_api_key='sk-proj-y8QksusK50Trrm0ZNKTnT3BlbkFJsloxsBlaoS7nGJWT8dva' openai_organization=None allowed_special=set() disallowed_special='all' chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None


In [35]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries",
                     embedding_function=OpenAIEmbeddings())

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

print(vectorstore)
print(retriever)

vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x11fee9ee0> docstore=<langchain_core.stores.InMemoryStore object at 0x11f663b90> search_kwargs={}


In [36]:
# List to store document entries for vector store
documents = []

In [59]:
import base64
import requests
import logging
import glob
import os

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Function to encode the image


def encode_image(image_path):
    logger.info(f"Encoding image from path: {image_path}")
    with open(image_path, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('utf-8')
    logger.info("Image encoding complete")
    return encoded_image


# Directory containing images
image_dir = "/Users/akileshjayakumar/Documents/catalystX2024/notebooks/mm_rag_images"

# Get all image files in the directory
image_paths = glob.glob(os.path.join(image_dir, "*.jpg"))

# List to store summaries
cleaned_img_summary = []

# Process each image
for image_path in image_paths:
    base64_image = encode_image(image_path)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}"
    }

    payload = {
        "model": "gpt-4o-mini",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "What’s in this image?"
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 300
    }

    logger.info(f"Sending request to OpenAI API for image: {image_path}")
    response = requests.post(
        "https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    if response.status_code == 200:
        logger.info("Request successful")
        summary = response.json().get('choices')[0]['message']['content']
        cleaned_img_summary.append(summary)
        print(summary)
    else:
        logger.error(f"Request failed with status code: {
                     response.status_code}")

INFO:__main__:Encoding image from path: /Users/akileshjayakumar/Documents/catalystX2024/notebooks/mm_rag_images/test1.jpg
INFO:__main__:Image encoding complete
INFO:__main__:Sending request to OpenAI API for image: /Users/akileshjayakumar/Documents/catalystX2024/notebooks/mm_rag_images/test1.jpg
INFO:__main__:Request successful
INFO:__main__:Encoding image from path: /Users/akileshjayakumar/Documents/catalystX2024/notebooks/mm_rag_images/test3.jpg
INFO:__main__:Image encoding complete
INFO:__main__:Sending request to OpenAI API for image: /Users/akileshjayakumar/Documents/catalystX2024/notebooks/mm_rag_images/test3.jpg


The image shows a math page from a workbook focused on finding the areas of rectangles. Here’s a summary of its contents:

1. **Math Message**: It asks to find the area of Rectangle A, which is given as 50 square units, with different expressions illustrating this area calculation.

2. **Rectangle B**: There are two number sentences written to express its area, calculated as 54 square units.

3. **Rectangle C**: The area is provided as 144 square units, with additional calculations involving a variable \( x \).

4. **Question 4**: Several mathematical expressions describe the areas of rectangles D, E, and F. Each expression is intended to be matched with one of the rectangles, while some matches have already been done.

Overall, the image illustrates a lesson on calculating and expressing the areas of rectangles using different methods and representations.


INFO:__main__:Request successful
INFO:__main__:Encoding image from path: /Users/akileshjayakumar/Documents/catalystX2024/notebooks/mm_rag_images/test2.jpg
INFO:__main__:Image encoding complete
INFO:__main__:Sending request to OpenAI API for image: /Users/akileshjayakumar/Documents/catalystX2024/notebooks/mm_rag_images/test2.jpg


The image contains a worksheet with educational activities. Here's a summary of its contents:

1. **Spelling Section**: Instructions for practicing spelling words from a language book, including a testing method using "look, say, cover."

2. **Math Section**: A math problem with missing numbers in a basic addition equation. It asks students to determine what the missing numbers could be. The equation sums to \( \frac{5}{6} \).

3. **Writing Section**: A prompt encouraging students to continue working on their Rotary Speech.

There are also illustrations alongside these sections.


INFO:__main__:Request successful


The image shows a worksheet with a question asking to find the area of a triangle. It includes dimensions: a base of 9 cm, a height of 5 cm, and there is a notation "A =" indicating where to write the area. The triangle appears to be part of a larger set of math problems, which includes references to other questions and possibly a context about someone named Jordan. The paper looks worn and crumpled.


In [61]:
import uuid
# Create documents from image summaries
documents = [Document(page_content=summary, metadata={
                      "doc_id": str(uuid.uuid4())}) for summary in cleaned_img_summary]

# Add documents to the vectorstore
vectorstore.add_documents(documents)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


['651406b2-a7b2-437a-9bdb-2ee34e3b0333',
 '77387a02-e5f9-4845-83f1-30119a8d9988',
 '7be0e795-2ccc-438e-a8c8-87b25ec8b751']

In [67]:
retriever = MultiVectorRetriever(vectorstore=vectorstore, docstore=store, id_key=id_key)
print(retriever)

vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x11fee9ee0> docstore=<langchain_core.stores.InMemoryStore object at 0x11f663b90> search_kwargs={}


In [86]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableSequence

# Prompt template
template = """
Answer the question based only on the following context: {context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Option 1: LLM
model = ChatOpenAI(temperature=0, model="gpt-4o")
# Option 2: Multi-modal LLM
# model = GPT4-V or LLaVA

# RAG pipeline
chain = (
    {"context": retriever, 
     "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print(chain)

first={
  context: MultiVectorRetriever(vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x11fee9ee0>, docstore=<langchain_core.stores.InMemoryStore object at 0x11f663b90>, search_kwargs={}),
  question: RunnablePassthrough()
} middle=[ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nAnswer the question based only on the following context: {context}\nQuestion: {question}\n'), additional_kwargs={})]), ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x14f13a7b0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x14aa8d9a0>, root_client=<openai.OpenAI object at 0x14ac951c0>, root_async_client=<openai.AsyncOpenAI object at 0x14f1505c0>, model_name='gpt-4o', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))]

In [87]:
response = chain.invoke("rertieve all you can")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [88]:
print(response)

I'm sorry, but there is no context provided for me to retrieve any information. Could you please provide more details or context?
